In [3]:
import pickle
import os
import time
import numpy as np
import os 
import shutil

import pyximport; pyximport.install()
from LEF_Dynamics import LEFTranslocatorDirectional

import warnings
import h5py 
import glob

## Defining simulation parameters

In [7]:

# -------defining parameters----------
#  -- basic loop extrusion parameters
SEPARATION = 200
LIFETIME = 300
N = 10000   # number of monomers
smcStepsPerBlock = 1  # now doing 1 SMC step per block 

trajectoryLength=100000


birthArray = np.zeros(N, dtype=np.double) + 0.1
deathArray = np.zeros(N, dtype=np.double) + 1. / LIFETIME
stallDeathArray = np.zeros(N, dtype=np.double) + 1 / LIFETIME
pauseArray = np.zeros(N, dtype=np.double)

stallList = [1000, 3000, 5000, 7000, 9000]
stallLeftArray = np.zeros(N, dtype = np.double)
stallRightARray = np.zeros(N, dtype = np.double)
for i in stallList:
    stallLeftArray[i] = 0.8
    stallRightARray[i] = 0.8

LEFNum = N // SEPARATION
LEFTran = LEFTranslocatorDirectional(birthArray, deathArray, stallLeftArray, stallRightARray, pauseArray,
                                     stallDeathArray, LEFNum)
folder = "trajectory"
if not os.path.exists(folder):
    os.mkdir(folder)
    
with h5py.File("trajectory/LEFTrajectory.h5", mode='w') as myfile:

    dset = myfile.create_dataset("trajectory", 
                                 shape=(trajectoryLength, LEFNum, 2), 
                                 dtype=np.int32, 
                                 compression="gzip")
    steps = 50 
    bins = np.linspace(0, trajectoryLength, steps, dtype=int)
    for st,end in zip(bins[:-1], bins[1:]):
        cur = []
        for i in range(st, end):
            LEFTran.steps(1)
            LEFs = LEFTran.getLEFs()
            cur.append(np.array(LEFs).T)
        cur = np.array(cur)
        dset[st:end] = cur
    myfile.attrs["N"] = N
    myfile.attrs["LEFNum"] = LEFNum




In [8]:
with h5py.File("trajectory/LEFTrajectory.h5", mode='r') as myfile:

    dset = myfile["trajectory"][:]
dset

array([[[9969, 9971],
        [6563, 6565],
        [2262, 2265],
        ...,
        [3704, 3707],
        [2363, 2366],
        [4974, 4977]],

       [[9968, 9972],
        [6562, 6566],
        [2261, 2266],
        ...,
        [3703, 3708],
        [2362, 2367],
        [4973, 4978]],

       [[9967, 9973],
        [6562, 6567],
        [2260, 2267],
        ...,
        [3702, 3709],
        [2361, 2368],
        [4972, 4979]],

       ...,

       [[7898, 7932],
        [4903, 4984],
        [1000, 2104],
        ...,
        [ 199,  465],
        [  32,  197],
        [4756, 4796]],

       [[7898, 7933],
        [4902, 4985],
        [1000, 2104],
        ...,
        [ 199,  465],
        [  31,  197],
        [4756, 4796]],

       [[7898, 7934],
        [4901, 4986],
        [1000, 2104],
        ...,
        [ 199,  465],
        [  30,  197],
        [4756, 4796]]], dtype=int32)